In [28]:
import requests
from pprint import pprint 
from time import time
from collections import Counter

import bs4
from bs4 import BeautifulSoup
from bs4.element import Comment

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.corpus import names as nltknames

import igraph
from igraph import Graph, EdgeSeq

stopwords = stopwords.words('english')
stopwords = [sw for sw in stopwords if len(sw)>1]
import json
from nameit import Nameit, contains, filter_names, count_freq_words



In [34]:
fout = open("stop_words.txt", 'w+', encoding='utf-8')
fout.writelines('\n'.join(stopwords))
fout.close()

In [45]:
last_names = []
with open('asian.txt') as f:
    asiannames = f.readlines()
last_names = [nm.strip().lower() for nm in asiannames]

with open('white.txt') as f:
    whitenames = f.readlines()
last_names.extend([nm.strip().lower() for nm in whitenames])

with open('top1000_lastnames.txt') as f:
    topnames = f.readlines()
last_names.extend([nm.strip().lower() for nm in topnames])

last_names.sort()
last_names = [ln for ln in last_names if ln!='']
last_names = list(set(last_names))
len(last_names)

3756

In [46]:
fout = open("last_names_list.txt", 'w+', encoding='utf-8')
fout.writelines('\n'.join(last_names))
fout.close()

In [47]:
first_names = [name.lower() for name in nltknames.words()]
first_names.sort()

fout = open("first_names_list.txt", 'w+', encoding='utf-8')
fout.writelines('\n'.join(first_names))
fout.close()

In [2]:
names = "get list of texts from different's fields".split()

names = [nm if not nm.endswith("'s") else nm[:-2] for nm in names]
names

['get', 'list', 'of', 'texts', 'from', 'different', 'fields']

In [16]:
#ref = 'https://www.nobelprize.org/prizes/uncategorized/all-nobel-prizes-in-physics/'
#ref = 'https://ballotpedia.org/List_of_current_members_of_the_U.S._Congress'
#ref = 'https://www.congress.gov/members'

#ref = 'https://chemistry.nd.edu/faculty-research/'
#ref = 'https://www.chem.wisc.edu/people/faculty' # 'Badger Chemist',
#ref = 'https://chbe.rice.edu/people/faculty'
#ref = 'https://www.chem.uci.edu/people/faculty'
ref = 'http://chemistry.mit.edu/people/faculty'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'}
page = requests.get(ref, headers = headers)
soup = BeautifulSoup(page.text, 'html.parser') # type bs4.BeautifulSoup



In [17]:
nmx = Nameit(soup)

In [18]:
nms = nmx.update_names()

In [19]:
#exclude stopwords from last and first names
sws = []
for sw in stopwords:
    if sw in nmx.first_names:
        print(sw)
        sws.append(sw)
    elif sw in nmx.last_names:
        print(sw + '  -- in last names')
        sws.append(sw)

for sw in sws:
    nmx.first_names = list(filter(lambda x: x != sw, nmx.first_names))
    nmx.last_names = list(filter(lambda x: x != sw, nmx.last_names))


you  -- in last names
he  -- in last names
her  -- in last names
do  -- in last names
an  -- in last names
to  -- in last names
in  -- in last names
on  -- in last names
no  -- in last names
so  -- in last names
than  -- in last names
will
don
haven
ma  -- in last names
shan  -- in last names
won  -- in last names


In [20]:
not_names = []
known_names = []
other_words = []

punct_marks = """,.:;!?"'"""

#for txt in nmx.visible_texts:
for txt in nmx.texts_from_tags:   
    txt = txt.translate(dict.fromkeys(map(ord, punct_marks), ' ')) 
    words = txt.lower().strip().split()
    words = [w.strip('-') for w in words]
    
    if 'hard' in txt.lower():
        print(txt)
        print(words)
        pf = True
    else:
        pf = False
    for w in words:
        
        if len(w) <=1:
            continue
        elif w[1]=='-':
            w = w[2:]
        
        if w in nmx.name_prefixes:
            continue
        elif w in (nmx.last_names + nmx.first_names):
            if pf: print('1 --', w)
            known_names.append(w)
            
        elif w in nmx.common_words:
            if pf: print('2 --', w)
            not_names.append(w)
        else:
            if pf: print('3 --', w)
            other_words.append(w)
            
            
n_limit = 5

freq_not_names, cntr = count_freq_words(not_names, n=1)

freq_known_names, cntr2 = count_freq_words(known_names, n=0)

freq_other_words, cntr3 = count_freq_words(other_words, n=4)

questionable_names = []
for wrd in freq_known_names:
    if wrd.lower() in nmx.common_words:
        questionable_names.append(wrd.lower())
        
too_often_names = []
for wfreq in cntr2.most_common():
    if (len(known_names)>100 and wfreq[1]>0.05*len(known_names)) or (len(known_names)<=100 and wfreq[1]>=5):
        too_often_names.append(wfreq[0])
        freq_known_names = list(filter(lambda x: x != wfreq[0], freq_known_names))
        

Schrock  Richard
['schrock', 'richard']
3 -- schrock
1 -- richard


In [21]:
#freq_not_names

In [22]:
#freq_not_names + too_often_names + freq_other_words

In [23]:
finalnames = []
questionable_names_list = []

xy = nmx.texts_from_tags
for it in xy:
    #if not contains(it, freq_known_names):
    if contains(it, freq_not_names + too_often_names + freq_other_words + stopwords):
        #print(it)
        continue
    if contains(it, questionable_names):
        questionable_names_list.append(filter_names(it))
        continue
    nm = filter_names(it)
    
    if 'Main' in it:
        print(it, filter_names(it))
    if nm!='':
        finalnames.append(nm)
finalnames = list(set(finalnames))
questionable_names_list = list(set(questionable_names_list))

In [24]:
#freq_not_names + too_often_names + freq_other_words + stopwords

In [25]:
questionable_names_list

['Van Voorhis Troy', 'Field Robert']

In [26]:
len(finalnames)
# Deficit Reduction',
# 'Job Opportunities'

59

In [27]:
finalnames

['Chakraborty Arup',
 'Swager Timothy',
 'Schrock Richard',
 'Poster Printer',
 'Nuclear Magnetic Resonance',
 'Materials Nanoscience',
 'Movassaghi Mohammad',
 'Raines Ronald',
 'Schlau-Cohen Gabriela',
 'Johnson Jeremiah',
 'Calendar Year Donors',
 'Chemical Biology',
 'Zhang Bin',
 'Cao Jianshu',
 'Danheiser Rick',
 'Photo Credits',
 'Fourier Transform Infrared Spectrometry',
 'Alumni Association',
 'Helpful Links',
 'Additional Outreach',
 'Kiessling Laura',
 'Mass Spectrometry',
 'Pentelute Bradley',
 'Electron Paramagnetic Resonance',
 'Finance Team',
 'Radosevich Alexander',
 'High Schools',
 'Suess Daniel',
 'Donor Profiles',
 'Maps Directions',
 'Administrative Assistants',
 'Drennan Catherine',
 'Willard Adam',
 'Site Map',
 'Dinca Mircea',
 'Nelson Keith',
 'Cummins Christopher',
 'Hong Mei',
 'Solomon Susan',
 'Buchwald Stephen',
 'Bawendi Moungi',
 'Ceyer Sylvia',
 'Community Relations',
 'Giving Alumni',
 'Online Application Instructions',
 'Imperiali Barbara',
 'Chemform

In [113]:
freq_not_names

['chemistry',
 'medicine',
 'life',
 'measurement',
 'energy',
 'organic',
 'faculty',
 'theory',
 'of',
 'search',
 'research',
 'university',
 'science',
 'department',
 'contact',
 'home',
 'graduate']

In [ ]:
nms.sort()
for nm in nms:
    print(nm)
   

In [ ]:
'charles' in 

In [117]:
len(words.words())

236736

In [77]:
ref = 'https://www.nobelprize.org/prizes/uncategorized/all-nobel-prizes-in-physics/'
ref2 = 'https://www.nobelprize.org/prizes/uncategorized/all-nobel-prizes-in-chemistry/'

In [79]:
nmprts = nameparts(cur_names, nltk_names, last_names)
    return cur_names, nmprts


In [71]:
ref = 'https://www.cies.org/fulbright-scholars'
ref = 'https://ballotpedia.org/United_States_Senate'
ref = 'https://ballotpedia.org/List_of_current_members_of_the_U.S._Congress'

In [87]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'}
page = requests.get(ref, headers = headers)
soup = BeautifulSoup(page.text, 'html.parser') # type bs4.BeautifulSoup



613 4
